<a href="https://colab.research.google.com/github/grantthomas/iops_resources/blob/master/demos/Keras_transfer_example_iops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!wget https://github.com/grantthomas/iops_resources/raw/master/keras-transfer-files/downloads.zip
#!unzip -n downloads.zip

#!wget https://github.com/grantthomas/iops_resources/raw/master/images/autolevel-small.zip
#!unzip -n autolevel-small.zip

In [2]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam


In [3]:

mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [4]:

base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [5]:

model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [6]:

for layer in model.layers:
    layer.trainable=True
# or if we want to set the first 20 layers of the network to be non-trainable
#for layer in model.layers[:20]:
    #layer.trainable=False
#for layer in model.layers[20:]:
    #layer.trainable=True

In [7]:

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./autolevel-small/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 800 images belonging to 2 classes.


In [8]:

def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor
  


In [9]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
25/25 [==============================] - 5s 184ms/step - loss: 0.4111 - accuracy: 0.9075
Epoch 2/10
25/25 [==============================] - 5s 191ms/step - loss: 0.0225 - accuracy: 0.9950
Epoch 3/10
25/25 [==============================] - 5s 190ms/step - loss: 0.0103 - accuracy: 0.9975
Epoch 4/10
25/25 [==============================] - 5s 191ms/step - loss: 0.0164 - accuracy: 0.9975
Epoch 5/10
25/25 [==============================] - 5s 193ms/step - loss: 0.0112 - accuracy: 0.9987
Epoch 6/10
25/25 [==============================] - 5s 193ms/step - loss: 0.0541 - accuracy: 0.9962
Epoch 7/10
25/25 [==============================] - 5s 194ms/step - loss: 0.0113 - accuracy: 0.9962
Epoch 8/10
25/25 [==============================] - 5s 195ms/step - loss: 0.0093 - accuracy: 0.9975
Epoch 9/10
25/25 [==============================] - 5s 195ms/step - loss: 0.0057 - accuracy: 0.9987
Epoch 10/10
25/25 [=====

In [19]:
img_path = './autolevel-small/test/false/camera-36_2020-10-25_21-45_to_2020-10-25_21-57.mp40661.jpg'
#expect [100, 0]
new_image = load_image(img_path)

pred = model.predict(new_image)

if (pred[0][0] > pred[0][1]):
  print("Eval: False")
else:
  print("Eval: True")

Eval: False


In [20]:
img_path = './autolevel-small/test/true/camera-41_2020-11-01_16-44_to_2020-11-01_16-56.mp40182.jpg'
#expect [0, 100]

new_image = load_image(img_path)

pred = model.predict(new_image)

if (pred[0][0] > pred[0][1]):
  print("Eval: False")
else:
  print("Eval: True")

Eval: True


In [21]:
img_path = './spirit-test-true.jpg'
#expect [0, 100]

new_image = load_image(img_path)

pred = model.predict(new_image)

if (pred[0][0] > pred[0][1]):
  print("Eval: False")
else:
  print("Eval: True")

Eval: True


In [23]:
import os

os.makedirs('./model', exist_ok=True)

model.save("./model/model.h5")